In [ ]:
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
from plotly import express as px

# Data Preprocessing
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score,
    train_test_split,
    GridSearchCV,
)

# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from imblearn.over_sampling import SMOTE, ADASYN

# Hyperparameter Tuning
import optuna

# Metrics
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    auc,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_auc_score,
    log_loss,
)

# Models
import xgboost as xgb

# Verifiy GPU
import cupy as cp

# Exporting model
import joblib

# <h1 style="font-size: 18px;">Import</h1>

In [ ]:
df_train = pd.read_csv("train.csv", index_col="id")

# <h1 style="font-size: 18px;">Statistics</h1>

In [ ]:
df_describe = df_train.describe()
df_describe.loc["IQR"] = df_describe.loc["75%"] - df_describe.loc["25%"]
df_describe

# <h1 style="font-size: 18px;">Density Plot</h1>

## <h1 style="font-size: 18px;">For each Rainfall Status</h1>

In [ ]:
def density_plot(df, col, target, ax):
    """Density plot for each column"""
    sns.kdeplot(df[df[target] == 0][col], ax=ax, color="blue", label="No Rainfall")
    sns.kdeplot(df[df[target] == 1][col], ax=ax, color="red", label="Rainfall")

    ax.axvline(df[col].mean(), color="black", linestyle="--", label="Mean")
    ax.axvline(df[col].median(), color="green", linestyle="--", label="Median")

    ax.legend()


# Create a figure with 5 rows and 2 columns
fig, axes = plt.subplots(figsize=(15, 25), nrows=5, ncols=2)
axes = axes.flatten()

# Plot each column in a separate subplot
for i, col in enumerate(df_train.drop(columns=["day", "rainfall"]).columns):
    density_plot(df_train, col, "rainfall", axes[i])
plt.suptitle("Density Plot for Each Rainfall Status", y=1, fontsize=20)
plt.tight_layout()
plt.show()

## <h1 style="font-size: 18px;">All</h1>

In [ ]:
def density_plot(df):

    fig, axes = plt.subplots(figsize=(15, 25), nrows=5, ncols=2)
    axes = axes.flatten()

    def density_plot(col, axes):
        """Density plot for each column"""
        sns.kdeplot(df[col], ax=axes, color="blue", label="Rainfall")

        axes.axvline(df[col].mean(), color="black", linestyle="--", label="Mean")
        axes.axvline(df[col].median(), color="green", linestyle="--", label="Median")

        axes.legend()

    # Plot each column in a separate subplot
    for i, col in enumerate(df.drop(columns=["day", "rainfall"]).columns):
        density_plot(col, axes[i])

    plt.suptitle("Density Plot All Data", y=1, fontsize=20)
    plt.tight_layout()
    plt.show()

In [ ]:
density_plot(df_train)

In [ ]:
df_train_scaled = df_train.copy()
df_train_scaled[df_train.drop(columns=["day", "rainfall"]).columns] = df_train.drop(columns=["day", "rainfall"]).apply(np.log1p)

density_plot(df_train_scaled)

# <h1 style="font-size: 18px;">Correlation</h1>

In [ ]:
# Correlation plot
plt.figure(figsize=(12, 6))
sns.heatmap(df_train.corr("spearman"), annot=True)
plt.title(
    "Correlation".upper(),
    fontdict={"family": "calibri", "fontsize": 18, "weight": "bold", "color": "red"},
    pad=20,
)
plt.show()

# <h1 style="font-size: 18px;">Matrix Scatter Plot</h1>

In [ ]:
# Scatter Plot
fig = px.scatter_matrix(
    df_train,
    dimensions=[col for col in df_train.drop(columns=["day", "rainfall"]).columns],
    color="rainfall"
)

fig.update_layout(
    title="Scatter Matrix",
    title_font_family="calibri",
    title_font_size=30,
    title_font_color="black",
    title_font_weight="bold",
    title_x=0.5,
    title_y=1,
    title_pad_t=20,
    height=1200,
    width=1200,
)

# Remover a barra de cores, mas manter a divisão de cores
fig.update_coloraxes(showscale=False)

fig.show()

# <h1 style="font-size: 18px;">BoxPlot</h1>

In [ ]:
fig = go.Figure()

cols = df_train.drop(columns=["day", "rainfall"]).columns
visible = [False] * (len(cols))

# Criando um boxplot para cada coluna, excluindo "day" e "rainfall"
for col in cols:
    fig.add_trace(
        go.Box(
            y=df_train[df_train["rainfall"] == 0][col],
            name=f"{col} - No Rainfall",
            visible=(col == cols[0]),
            hovertemplate="<b>%{x}:</b> %{y:.2f}<extra></extra>",
        )
    )

    fig.add_trace(
        go.Box(
            y=df_train[df_train["rainfall"] == 1][col],
            name=f"{col} - Rainfall",
            visible=(col == cols[0]),
            hovertemplate="<b>%{x}:</b> %{y:.2f}<extra></extra>",
        )
    )

buttons =[
    dict(
        label=col,
        method="update",
        args=[
            {"visible": [col == trace.name.split(" - ")[0] for trace in fig.data]},
            ]
        ) for col in cols
    ]

# Configurações do gráfico
fig.update_layout(
    height=600,
    width=1300,
    title={
        "text": "BoxPlot",  # Título do gráfico
        "y": 0.98,  # Posição vertical do título (0 a 1)
        "x": 0.5,  # Posição horizontal do título (0 a 1)
        "xanchor": "center",  # Ancoragem horizontal
        "yanchor": "top",  # Ancoragem vertical
        "font": {
            "family": "Arial, sans-serif",
            "size": 26,
            "color": "black",
            "weight": "bold",
        },
    },
    margin=dict(l=50, r=20, t=100, b=0),  # Margem da área de plotagem
    updatemenus=[  # Botões do Filtro. Método menos eficiente
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.5,  # Posição horizontal dos botões (0 a 1)
            y=1.1,  # Posição vertical dos botões (0 a 1)
            xanchor="center",
            yanchor="top",
            active=0,
        )
    ],
    showlegend=False,  # Ocultar a legenda
)

# Configurar eixo X
fig.update_xaxes(
    title_text="",
    tickfont=dict(family="Arial", size=12, color="black", weight="bold"),
)

# Configurar eixo Y
fig.update_yaxes(
    title_text="",
    tickfont=dict(family="Arial", size=14, color="black", weight="bold"),
    tickformat=".0f",
)

fig.show()

# <h1 style="font-size: 18px;">Model ML</h1>

In [ ]:
# Check if GPU is available
try:
    cuda_available = cp.cuda.runtime.getDeviceCount() > 0
except Exception:
    cuda_available = False

# Set tree method
tree_method = "gpu_hist" if cuda_available else "hist"
device = "cuda" if cuda_available else "cpu"

print(f"Device: {device}")
print(f"Tree Method: {tree_method}")

In [ ]:

# Split data into features and target
X = df_train.drop(columns=["rainfall"])
Y = df_train["rainfall"]

# Split data into train and test
X1, X2, Y1, Y2 = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Resample data
X1_resampled, Y1_resampled = ADASYN(random_state=42,n_neighbors=7).fit_resample(X1, Y1)

#X1_resampled, Y1_resampled = SMOTE(random_state=42).#fit_resample(X1, Y1)

In [ ]:
# Train model with Hyperparameter Tuning
def objective(trial):
    
    # Parameters
    params = {
        "device": device, # Use GPU
        "tree_method":tree_method, # Optimize for GPU
        "objective": "binary:logistic",  # Binary Classification
        "eval_metric": "auc", # Metric to evaluate
        "random_state": 42, # Random seed
        "verbosity": 0, # Verbosity of printing messages
        #"scale_pos_weight": Y1[Y1 == 0].shape[0] / Y1[Y1 == 1].shape[0], # Control the balance of positive and negative weights
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),  # Number of trees
        "max_depth": trial.suggest_int("max_depth", 1, 10), # Maximum depth of the tree
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),  # Learning rate
        "subsample": trial.suggest_float("subsample", 0.1, 1), # Subsample ratio of the training instances
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1), 
        "gamma": trial.suggest_float("gamma", 0, 1), # Minimum loss reduction required to make a further partition on a leaf node of the tree
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),  # Minimum sum of instance weight (hessian) needed in a child
        "reg_alpha": trial.suggest_float("reg_alpha", 0.001, 0.1),  # L1 regularization term on weights
        "reg_lambda": trial.suggest_float("reg_lambda", 0.001, 0.1),  # L2 regularization term on weights
        "early_stopping_rounds": 20, # Early stopping
        "verbose": False,
    }
    
    auc_scores = [] # List to store AUC scores

    model = xgb.XGBClassifier(**params) # Create model

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # Cross Validation
    
    # Train model
    for train_index, test_index in cv.split(X1_resampled, Y1_resampled):
        X_train, X_test = X1_resampled.iloc[train_index], X1_resampled.iloc[test_index]
        Y_train, Y_test = Y1_resampled.iloc[train_index], Y1_resampled.iloc[test_index]
        
        model.fit(
            X_train,
            Y_train,
            eval_set=[(X_test, Y_test)], # Validation set
            verbose=False,
        )
    
        Y_pred = model.predict(X_test)
        Y_proba = model.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(Y_test, Y_proba)
        auc_scores.append(auc_score)
    
    return np.mean(auc_scores)

# Create study
study = optuna.create_study(direction="maximize", study_name="Rainfall_1") # Maximize AUC score
study.optimize(objective, n_trials=100, show_progress_bar=True) # Optimize study

In [ ]:
study.best_params

In [ ]:
trial = study.best_trial
print(f"Best Trial: {trial.number}")
print(f"ROC AUC: {trial.value:.4f}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value:.4f}")

In [ ]:
# Plot Optimization History
optuna.visualization.plot_optimization_history(study).show()

In [ ]:
# Plot Parameter Importances
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
# Train model with best parameters
param = {
    "device": "cuda",
    "tree_method": "hist",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "random_state": 42,
    "verbosity": 0,
    "scale_pos_weight": Y1[Y1 == 0].shape[0] / Y1[Y1 == 1].shape[0],
    "early_stopping_rounds": 20,
    "verbose": False,
    **trial.params,
}

model = xgb.XGBClassifier(**param)
model.fit(X1, Y1, eval_set=[(X2, Y2)], verbose=False)

# Predict
Y_pred = model.predict(X2)
Y_proba = model.predict_proba(X2)[:, 1]

# Metrics
accuracy = accuracy_score(Y2, Y_pred)
precision = precision_score(Y2, Y_pred)
recall = recall_score(Y2, Y_pred)
f1 = f1_score(Y2, Y_pred)
roc_auc = roc_auc_score(Y2, Y_proba)
logloss = log_loss(Y2, Y_proba)

print("Trained Model:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Log Loss: {logloss:.4f}")

In [ ]:
print("Trained Model:".center(20, "-"))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Log Loss: {logloss:.4f}")

In [ ]:
# Feature Importance
feature_importance = model.feature_importances_
feature_importance = np.round(feature_importance * 100, 3)

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=X.columns,
        y=feature_importance,
        marker_color="blue",
        hovertemplate="<b>%{x}:</b> %{y:.2f}%<extra></extra>",
    )
)

fig.update_layout(
    title="Feature Importance",
    title_font_family="calibri",
    title_font_size=30,
    title_font_color="black",
    title_font_weight="bold",
    title_x=0.5,
    title_y=0.95,
    title_pad_t=20,
    xaxis_title="Features",
    yaxis_title="Importance",
    xaxis_tickangle=-45,
)

fig.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(Y2, Y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(Y2, Y_proba) # False Positive Rate, True Positive Rate, Thresholds
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color="red", label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="blue", linestyle="--")
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
# Classification Report
print(classification_report(Y2, Y_pred))

In [ ]:
# Plot Metrics by Threshold
metrics = []

for threshold in thresholds:
    Y_pred_threshold = (Y_proba > threshold).astype(int)
    accuracy = accuracy_score(Y2, Y_pred_threshold)
    precision = precision_score(Y2, Y_pred_threshold, zero_division=0)
    recall = recall_score(Y2, Y_pred_threshold)
    f1 = f1_score(Y2, Y_pred_threshold)
    roc_auc = roc_auc_score(Y2, Y_proba)
    logloss = log_loss(Y2, Y_proba)
    metrics.append([accuracy, precision, recall, f1, roc_auc, logloss])

metrics = np.array(metrics)

fig = go.Figure()

fig.add_trace(go.Scatter(x=thresholds, y=metrics[:, 0], mode="lines", name="Accuracy"))
fig.add_trace(go.Scatter(x=thresholds, y=metrics[:, 1], mode="lines", name="Precision"))
fig.add_trace(go.Scatter(x=thresholds, y=metrics[:, 2], mode="lines", name="Recall"))
fig.add_trace(go.Scatter(x=thresholds, y=metrics[:, 3], mode="lines", name="F1"))
fig.add_trace(go.Scatter(x=thresholds, y=metrics[:, 4], mode="lines", name="ROC AUC"))
fig.add_trace(go.Scatter(x=thresholds, y=metrics[:, 5], mode="lines", name="Log Loss"))

fig.update_layout(
    title="Metrics by Threshold (ROC Curve)",
    title_x=0.5,
    title_font_size=20,
    title_font_weight="bold",
    xaxis_title="Threshold",
    yaxis_title="Value",
)

fig.show()

In [ ]:
# Metric with Optimal Threshold
optimal_threshold = thresholds[np.argmax(metrics[:, 3])] # F1 Score
Y_pred_optimal = (Y_proba > optimal_threshold).astype(int)  # Predict

accuracy = accuracy_score(Y2, Y_pred_optimal)
precision = precision_score(Y2, Y_pred_optimal, zero_division=0)
recall = recall_score(Y2, Y_pred_optimal)
f1 = f1_score(Y2, Y_pred_optimal)
roc_auc = roc_auc_score(Y2, Y_proba)
logloss = log_loss(Y2, Y_proba)

print("Optimal Threshold:".center(40, "-"))
print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Log Loss: {logloss:.4f}")

# Confusion Matrix by Optimal Threshold
cm = confusion_matrix(Y2, Y_pred_optimal)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix by Optimal Threshold")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# <h1 style="font-size: 18px;">Aplicação ML no dataframe de test</h1>

In [ ]:
df_test = pd.read_csv("test.csv", index_col="id")

y_pred_test = model.predict(df_test)
y_pred_proba_test = model.predict_proba(df_test)[:, 1]

df_test["rainfall"] = y_pred_test
df_test["rainfall_proba"] = y_pred_proba_test
df_test["rainfall_proba"] = df_test["rainfall_proba"].apply(lambda x: round(x * 100, 2))

df_test[["rainfall", "rainfall_proba"]].to_csv("submission.csv", index=True)